### Load necessary libraries

In [1]:
from shapely.geometry import shape, mapping
from shapely.ops import unary_union


import json

### Read the dataset from the GeoJSON file

In [2]:
with open('data/romania-with-regions.geojson', 'r') as file:
    geojson_data = json.load(file)

### Group counties into historical regions

#### Define the list of counties for each region

In [3]:
# List of regions and their corresponding counties
regions = {
    "Muntenia": ["Prahova", "Buzau", "Arges", "Dambovita", "Ilfov", "Bucuresti", "Teleorman", "Giurgiu", "Calarasi", "Ialomita", "Braila", "Buzau"],
    "Dobrogea": ["Tulcea", "Constanta"],
    "Moldova": ["Suceava", "Botosani", "Iasi", "Neamt", "Bacau", "Vaslui", "Vrancea", "Galati"],
    "Transilvania": ["Satu Mare", "Maramures", "Bistrita-Nasaud", "Salaj", "Bihor", "Arad", "Cluj", "Mures", "Harghita", "Covasna", "Brasov", "Sibiu", "Alba", "Hunedoara"],
    "Banat": ["Timis", "Caras-Severin"],
    "Oltenia": ["Mehedinti", "Gorj", "Valcea", "Olt", "Dolj"]
}

# Function to create region features
def create_region_feature(region_name, county_names):
    # Collect the features for the specified counties in the region
    region_features = []
    for feature in geojson_data['features']:
        if feature['properties']['name'] in county_names:
            region_features.append(shape(feature['geometry']))
    # Merge geometries of the counties
    region_shape = unary_union(region_features)
    # Create a new feature for the region
    region_feature = {
        'type': 'Feature',
        'properties': {
            'name': region_name
        },
        'geometry': mapping(region_shape)
    }
    return region_feature

In [4]:
all_region_features = [create_region_feature(region, counties) for region, counties in regions.items()]

# Create a new GeoJSON object for the updated map with regions
all_regions_geojson = {
    'type': 'FeatureCollection',
    'features': all_region_features
}

with open('data/regions_final.geojson', 'w') as new_file:
    json.dump(all_regions_geojson, new_file)
